# Projet DM

In [1]:
!pip install unidecode
!pip install spacy
!python -m spacy download fr_core_news_sm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
     - -------------------------------------- 0.5/16.3 MB 2.1 MB/s eta 0:00:08
     --- ------------------------------------ 1.3/16.3 MB 2.9 MB/s eta 0:00:06
     ----- ---------------------------------- 2.1/16.3 MB 3.2 MB/s eta 0:00:05
     ------- -------------------------------- 3.1/16.3 MB 3.7 MB/s eta 0:00:04
     --------- ------------------------------ 3.9/16.3 MB 3.8 MB/s eta 0:00:04
     ------------- -------------------------- 5.5/16.3 MB 4.4 MB/s eta 0:00:03
     ------------------ --------------------- 7.3/16.3 MB 5.0 MB/s eta 0:00:02
     --------------------- ------------------ 8.9/16.3 MB 5.3 MB/s eta 0:00:02
     --------------------------- ------------ 11.0/16.3 MB 5.9 MB/s eta 0:00:01
     ------------------------------- -------- 12.8/16.3 MB 6.3 MB/s eta 0:00:01
     -------------------------------------- - 15.7/16.3 MB 6.9 M


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import os
import pandas as pd
import re
from unidecode import unidecode

def load_jsonl(filename):
    """Load a .jsonl file. Tries both the given path and the 'data' folder.
    Returns a list of dicts."""
    p = os.path.join('data', filename)
    if os.path.exists(p):
        data = []
        with open(p, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    data.append(json.loads(line))
                except Exception as e:
                    print(f"Error parsing a line in {p}: {e}")
        return data
    raise FileNotFoundError(f"File not found: {filename} (tried: {p})")

### 1. Data Upload

In [3]:
# Load the data 
train_path = 'train_v2.jsonl'
test_path = 'test_v4.jsonl'

train_data = load_jsonl(train_path)
test_data = load_jsonl(test_path)

print(f"Train records: {len(train_data)}, Test records: {len(test_data)}")

# Convert to DataFrame for easier handling
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Quick preview
print("-- Train head --")
print(train_df.head())
print("\n-- Info --")
print(train_df.info())

Train records: 492, Test records: 519
-- Train head --
                                                text acronym  \
0  LRA  limite de résistance des attelages PAR po...     PAR   
1                              Désigna -tion des PN       PN   
2  prédéterminées de trains : _x0001_ les masses ...      EM   
3  /Commentaires N° AC B81500 thermique:  compati...      AC   
4  kilomètres/heure (ex : 12 pour 120 km/h), _x00...     TIV   

                                             options  
0  {'Plan d'action régularité': False, 'Poste d'a...  
1  {'Passages à niveau : fichier des pn, recensem...  
2  {'EMERAINVILLE PONTAULT COMBAULT': False, 'Eng...  
3  {'ACcès': False, 'Agent d'aCcompagnement ': Fa...  
4  {'THIVIERS': False, 'Trafic international voya...  

-- Info --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     492 non-null    object
 1 

### 2. Data Cleaning

In [4]:
# Basic cleaning: columns, missing values, duplicates
print("Columns:", list(train_df.columns))
print("Missing values per column:\n", train_df.isna().sum())

train_df = train_df.drop_duplicates(subset=['text', 'acronym']).reset_index(drop=True)

for col in train_df.select_dtypes(include='object').columns:
    train_df[col] = train_df[col].fillna('')

print("After cleaning, shape:", train_df.shape)

Columns: ['text', 'acronym', 'options']
Missing values per column:
 text       0
acronym    0
options    0
dtype: int64
After cleaning, shape: (492, 3)


### 3. Pre-Processing

In [5]:
import nltk
import spacy
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources (once)
nltk.download("punkt")
nltk.download("stopwords")

# Load spaCy French model
nlp = spacy.load("fr_core_news_sm")

stop_words = set(stopwords.words("french"))

def preprocessing(text, options=False):
    """Unified preprocessing for text and options."""
    if pd.isna(text) or text is None:
        return ""
    text = str(text).lower()
    text = unidecode(text)
    try:
        tokens = word_tokenize(text, language='french')
    except:
        tokens = text.split()
    if options:
        filtered_tokens = [w for w in tokens if w.isalpha()]
    else:
        filtered_tokens = [w for w in tokens if w not in stop_words and w.isalpha()]
    text_to_lemmatize = " ".join(filtered_tokens)
    try:
        doc = nlp(text_to_lemmatize)
        lemmas = [token.lemma_ for token in doc if token.is_alpha]
        return " ".join(lemmas)
    except Exception as e:
        print(f"Lemmatization error: {e}")
        return text_to_lemmatize

print("Processing train texts...")
train_df['text_processed'] = train_df['text'].apply(preprocessing)
print("Processing test texts...")
test_df['text_processed'] = test_df['text'].apply(preprocessing)

def process_options_dict(options_dict):
    """Process training options (dict)."""
    return {preprocessing(k, True): v for k, v in options_dict.items()}

def process_options_list(options_list):
    """Process test options (list)."""
    return [preprocessing(opt, True) for opt in options_list]

print("Processing train options...")
train_df['options_processed'] = train_df['options'].apply(process_options_dict)
print("Processing test options...")
test_df['options_processed'] = test_df['options'].apply(process_options_list)

print("✅ Preprocessing completed")
print(f"Sample processed text: {train_df['text_processed'].iloc[0][:100]}...")
print(f"Sample processed options keys: {list(train_df['options_processed'].iloc[0].keys())}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing train texts...
Processing test texts...
Processing test texts...
Processing train options...
Processing train options...
Processing test options...
Processing test options...
✅ Preprocessing completed
Sample processed text: lra limite resistance attelage poste regulation pl pleine lign pn passage avoir niveau rfn reseau fe...
Sample processed options keys: ['plan regularite', 'poste et de regulation assurer le commande de installation de signalisation et le gestion de le circulation de huit ligne avoir grand vitesse', 'pont de', 'plan regional']
✅ Preprocessing completed
Sample processed text: lra limite resistance attelage poste regulation pl pleine lign pn passage avoir niveau rfn reseau fe...
Sample processed options keys: ['plan regularite', 'poste et de regulation assurer le commande de installation de signalisation et le gestion de le circulation de huit ligne avoir grand vitesse', 'pont de', 'plan regional']


### 5. Training

### TF-IDF

In [6]:
from sklearn.model_selection import train_test_split
train_final, val_final = train_test_split(train_df, test_size=0.3, random_state=100)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("Building TF-IDF vocabulary...")
all_texts_for_tfidf = []
all_texts_for_tfidf.extend(train_final['text_processed'].tolist())
for options_dict in train_final['options_processed']:
    all_texts_for_tfidf.extend(options_dict.keys())
all_texts_for_tfidf.extend(test_df['text_processed'].tolist())

global_tfidf = TfidfVectorizer(ngram_range=(1,3), min_df=2, max_features=10000)
global_tfidf.fit(all_texts_for_tfidf)
print(f"✅ Global TF-IDF trained with {len(global_tfidf.vocabulary_)} terms")

Building TF-IDF vocabulary...
✅ Global TF-IDF trained with 7425 terms


In [8]:
def get_answer_index(row):
    options = row['options_processed']
    for idx, option in enumerate(options):
        if options[option] == True:
            return idx
    return -1
val_final['answer_index'] = val_final.apply(get_answer_index, axis=1)
val_final.head()

,text,acronym,options,text_processed,options_processed,answer_index
184,VENISSIEUX à AMBERIEU Article B101 Domaine de...,EF,{'Entreprise Ferroviaire : Toute entreprise ...,venissieux avoir amberieu article domaine circ...,{'entreprise ferroviaire tout entreprise avoir...,0
252,Boingneville PLBV – Poste 1MALESHERBES(1) – Et...,BV,"{'Bassin Versant': False, 'Bâtiment des Voyage...",boingnevill plbv poste etablissement pl suscep...,"{'bassin verser': False, 'batiment de voyageur...",1
397,Feuquières Fressenneville - PL 30Woincourt - P...,PL,"{'Panneaux lumineux': False, 'Pleine Ligne. ...",feuquiere fressennevill pl pl kilomètre ligne ...,"{'panneau lumineux': False, 'pleine lign etabl...",-1
325,franchissement des signaux d'arrêt Ligne équip...,DAAT,"{'Diagnostique Amiante Avant Travaux': False, ...",franchissemer signal ligne equipe dispositif a...,"{'diagnostique amiant avant travail': False, '...",1
136,Page 4 NPDC-RT-2211B- Version 01 du 15-12-2014...,EF,"{'Equipement fixe': False, 'Essai de Frein': F...",page version article domaine circulation appar...,"{'equipement fixe': False, 'essai de frein': F...",2


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def improved_similarity_prediction_validation(df, threshold=0.01, ngram_range=(1, 2),
                                            use_acronym=True, query_strategy="text_acronym",
                                            fallback_strategy="first"):
    """Improved similarity-based prediction for validation."""
    predictions = []
    validation_scores = []
    confidence_scores = []

    for idx, row in df.iterrows():
        try:
            text = row['text_processed']
            acronym = row['acronym']
            options = row['options_processed']

            if query_strategy == "text_only":
                query = text
            elif query_strategy == "acronym_only":
                query = acronym
            elif query_strategy == "text_acronym":
                query = f"{text} {acronym}"
            elif query_strategy == "acronym_text":
                query = f"{acronym} {text}"
            elif query_strategy == "weighted_text":
                query = f"{text} {text} {acronym}"
            else:
                query = f"{text} {acronym}"

            vectorizer = TfidfVectorizer(ngram_range=ngram_range)
            options_list = list(options.keys())
            corpus = [query] + options_list
            tfidf_matrix = vectorizer.fit_transform(corpus)
            similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

            best_idx = np.argmax(similarities)
            best_score = similarities[best_idx]

            confidence_scores.append(best_score)

            if best_score > threshold:
                validation_scores.append(1 if best_idx == row['answer_index'] else 0)
                predictions.append([best_idx])
            else:
                if fallback_strategy == "first":
                    fallback_idx = 0
                elif fallback_strategy == "random":
                    fallback_idx = np.random.randint(0, len(options_list))
                elif fallback_strategy == "last":
                    fallback_idx = len(options_list) - 1
                elif fallback_strategy == "middle":
                    fallback_idx = len(options_list) // 2
                else:
                    fallback_idx = 0
                predictions.append([fallback_idx])
                validation_scores.append(1 if fallback_idx == row['answer_index'] else 0)

        except Exception as e:
            print(f"Row {idx} error: {e}")
            predictions.append([0])
            validation_scores.append(0)

    accuracy = np.mean(validation_scores) if validation_scores else 0
    avg_confidence = np.mean(confidence_scores) if confidence_scores else 0
    coverage = np.mean([1 if score > threshold else 0 for score in confidence_scores]) if confidence_scores else 0

    return predictions, accuracy, avg_confidence, coverage

print("=== FULL VALIDATION WITH MULTIPLE PARAMETERS ===\n")

thresholds = [0, 0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25]
for thresh in thresholds:
    _, accuracy, avg_conf, coverage = improved_similarity_prediction_validation(val_final, threshold=thresh)
    print(f"  Threshold {thresh:.3f}: Accuracy={accuracy:.4f}, Avg_conf={avg_conf:.4f}, Coverage={coverage:.4f}")

ngram_ranges = [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)]
for ngram in ngram_ranges:
    _, accuracy, avg_conf, coverage = improved_similarity_prediction_validation(val_final, ngram_range=ngram)
    print(f"  ngram_range {ngram}: Accuracy={accuracy:.4f}, Avg_conf={avg_conf:.4f}, Coverage={coverage:.4f}")

query_strategies = ["text_only", "acronym_only", "text_acronym", "acronym_text", "weighted_text"]
for strategy in query_strategies:
    _, accuracy, avg_conf, coverage = improved_similarity_prediction_validation(val_final, query_strategy=strategy)
    print(f"  Query '{strategy}': Accuracy={accuracy:.4f}, Avg_conf={avg_conf:.4f}, Coverage={coverage:.4f}")

fallback_strategies = ["first", "random", "last", "middle"]
for fallback in fallback_strategies:
    _, accuracy, avg_conf, coverage = improved_similarity_prediction_validation(val_final, fallback_strategy=fallback)
    print(f"  Fallback '{fallback}': Accuracy={accuracy:.4f}, Avg_conf={avg_conf:.4f}, Coverage={coverage:.4f}")

combinations = [
    {"threshold": 0.01, "ngram_range": (1, 2), "query_strategy": "text_acronym"},
    {"threshold": 0.05, "ngram_range": (1, 3), "query_strategy": "weighted_text"},
    {"threshold": 0.02, "ngram_range": (1, 2), "query_strategy": "text_only"},
    {"threshold": 0.1, "ngram_range": (2, 3), "query_strategy": "acronym_text"},
]
for i, combo in enumerate(combinations):
    _, accuracy, avg_conf, coverage = improved_similarity_prediction_validation(val_final, **combo)
    print(f"  Combo {i+1}: Accuracy={accuracy:.4f}, Avg_conf={avg_conf:.4f}, Coverage={coverage:.4f}")
    print(f"    Params: {combo}")

fine_thresholds = [0.005, 0.008, 0.01, 0.012, 0.015, 0.018, 0.02, 0.025]
best_threshold = 0
best_accuracy = 0
for thresh in fine_thresholds:
    _, accuracy, avg_conf, coverage = improved_similarity_prediction_validation(val_final, threshold=thresh)
    print(f"  Threshold {thresh:.3f}: Accuracy={accuracy:.4f}, Avg_conf={avg_conf:.4f}, Coverage={coverage:.4f}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = thresh
print(f"\n🎯 BEST THRESHOLD: {best_threshold} with accuracy: {best_accuracy:.4f}")

def find_best_parameters(validation_df, n_trials=50):
    """Random search for best params."""
    best_params = {}
    best_accuracy = 0
    thresholds = [0, 0.001, 0.005, 0.01, 0.02, 0.05, 0.1]
    ngram_ranges = [(1,1),(1,2),(1,3),(2,2),(2,3)]
    query_strategies = ["text_only","acronym_only","text_acronym","weighted_text"]
    for i in range(n_trials):
        threshold = np.random.choice(thresholds)
        ngram_range = ngram_ranges[np.random.randint(len(ngram_ranges))]
        query_strategy = query_strategies[np.random.randint(len(query_strategies))]
        try:
            _, accuracy, _, _ = improved_similarity_prediction_validation(validation_df, threshold=threshold, ngram_range=ngram_range, query_strategy=query_strategy)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'threshold': threshold, 'ngram_range': ngram_range, 'query_strategy': query_strategy, 'accuracy': accuracy}
                print(f"🔥 New best: Accuracy={accuracy:.4f}, threshold={threshold}, ngram_range={ngram_range}, query_strategy='{query_strategy}'")
        except Exception as e:
            print(f"Trial {i} error: {e}")
            continue
    return best_params

print("\n7. RANDOM PARAM SEARCH:")
best_params = find_best_parameters(val_final, n_trials=30)
print("\n🏆 BEST PARAMETERS FOUND:")
for key, value in best_params.items():
    print(f"  {key}: {value}")

if best_params:
    print("\n8. FINAL EVAL WITH BEST PARAMETERS:")
    final_predictions, final_accuracy, final_confidence, final_coverage = improved_similarity_prediction_validation(val_final, threshold=best_params['threshold'], ngram_range=best_params['ngram_range'], query_strategy=best_params['query_strategy'])
    print(f"✅ Final result: Accuracy={final_accuracy:.4f}")

=== FULL VALIDATION WITH MULTIPLE PARAMETERS ===

  Threshold 0.000: Accuracy=0.5068, Avg_conf=0.0606, Coverage=0.7500
  Threshold 0.000: Accuracy=0.5068, Avg_conf=0.0606, Coverage=0.7500
  Threshold 0.001: Accuracy=0.5068, Avg_conf=0.0606, Coverage=0.7500
  Threshold 0.001: Accuracy=0.5068, Avg_conf=0.0606, Coverage=0.7500
  Threshold 0.005: Accuracy=0.5068, Avg_conf=0.0606, Coverage=0.7500
  Threshold 0.005: Accuracy=0.5068, Avg_conf=0.0606, Coverage=0.7500
  Threshold 0.010: Accuracy=0.5000, Avg_conf=0.0606, Coverage=0.7230
  Threshold 0.010: Accuracy=0.5000, Avg_conf=0.0606, Coverage=0.7230
  Threshold 0.020: Accuracy=0.4595, Avg_conf=0.0606, Coverage=0.6554
  Threshold 0.020: Accuracy=0.4595, Avg_conf=0.0606, Coverage=0.6554
  Threshold 0.050: Accuracy=0.3581, Avg_conf=0.0606, Coverage=0.4392
  Threshold 0.050: Accuracy=0.3581, Avg_conf=0.0606, Coverage=0.4392
  Threshold 0.100: Accuracy=0.2973, Avg_conf=0.0606, Coverage=0.1959
  Threshold 0.100: Accuracy=0.2973, Avg_conf=0.0606, 

In [12]:
### 6. Final test prediction

def improved_similarity_prediction_test(df, threshold=0.01, ngram_range=(1, 2),
                                      query_strategy="text_acronym", fallback_strategy="first"):
    """Produce predictions for the test set."""
    predictions = []
    confidence_scores = []
    
    for idx, row in df.iterrows():
        try:
            text = row['text_processed']
            acronym = row['acronym']
            options = row['options_processed']  # test options list
            
            if query_strategy == "text_only":
                query = text
            elif query_strategy == "acronym_only":
                query = acronym
            elif query_strategy == "text_acronym":
                query = f"{text} {acronym}"
            elif query_strategy == "acronym_text":
                query = f"{acronym} {text}"
            elif query_strategy == "weighted_text":
                query = f"{text} {text} {acronym}"
            else:
                query = f"{text} {acronym}"

            vectorizer = TfidfVectorizer(ngram_range=ngram_range)
            options_list = options
            corpus = [query] + options_list
            tfidf_matrix = vectorizer.fit_transform(corpus)
            similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

            best_idx = np.argmax(similarities)
            best_score = similarities[best_idx]

            confidence_scores.append(best_score)

            if best_score > threshold:
                predictions.append([best_idx])
            else:
                if fallback_strategy == "first":
                    fallback_idx = 0
                elif fallback_strategy == "random":
                    fallback_idx = np.random.randint(0, len(options_list))
                elif fallback_strategy == "last":
                    fallback_idx = len(options_list) - 1
                elif fallback_strategy == "middle":
                    fallback_idx = len(options_list) // 2
                else:
                    fallback_idx = 0
                predictions.append([fallback_idx])
                    
        except Exception as e:
            print(f"Row {idx} error: {e}")
            predictions.append([0])
            confidence_scores.append(0)
    
    avg_confidence = np.mean(confidence_scores) if confidence_scores else 0
    coverage = np.mean([1 if score > threshold else 0 for score in confidence_scores]) if confidence_scores else 0
    
    return predictions, avg_confidence, coverage

print("=== MAKING FINAL TEST PREDICTIONS ===")

if 'best_params' in locals() and best_params:
    print(f"Using best params: {best_params}")
    test_predictions, test_confidence, test_coverage = improved_similarity_prediction_test(
        test_df,
        threshold=best_params['threshold'],
        ngram_range=best_params['ngram_range'],
        query_strategy=best_params['query_strategy']
    )
else:
    print("Using default params")
    test_predictions, test_confidence, test_coverage = improved_similarity_prediction_test(
        test_df,
        threshold=0.01,
        ngram_range=(1, 2),
        query_strategy="text_acronym"
    )

print(f"✅ Test predictions done: {len(test_predictions)} items")
print(f"   - Avg confidence: {test_confidence:.4f}")
print(f"   - Coverage: {test_coverage:.4f}")

predictions_df = pd.DataFrame({
    'id': test_df['id'],
    'answer': [pred for pred in test_predictions]
})

print("\nPredicted answer distribution:")
print(predictions_df['answer'].value_counts().sort_index())

output_filename = 'submission_improved.csv'
predictions_df.to_csv(output_filename, index=False)
print(f"\nSaved submission: {output_filename}")

print("\n=== CHECK SAVED FILE ===")
saved_df = pd.read_csv(output_filename)
print(f"Loaded file rows: {saved_df.shape[0]}")
print("First 5 rows:")
print(saved_df.head())

=== MAKING FINAL TEST PREDICTIONS ===
Using best params: {'threshold': 0.001, 'ngram_range': (1, 2), 'query_strategy': 'text_only', 'accuracy': 0.527027027027027}
✅ Test predictions done: 519 items
   - Avg confidence: 0.0449
   - Coverage: 0.6686

Predicted answer distribution:
answer
[0]     258
[1]     119
[2]      63
[3]      62
[4]       5
[5]       2
[6]       3
[7]       3
[8]       1
[9]       2
[12]      1
Name: count, dtype: int64

Saved submission: submission_improved.csv

=== CHECK SAVED FILE ===
Loaded file rows: 519
First 5 rows:
   id answer
0   0    [1]
1   1    [1]
2   2    [0]
3   3    [1]
4   4    [0]
✅ Test predictions done: 519 items
   - Avg confidence: 0.0449
   - Coverage: 0.6686

Predicted answer distribution:
answer
[0]     258
[1]     119
[2]      63
[3]      62
[4]       5
[5]       2
[6]       3
[7]       3
[8]       1
[9]       2
[12]      1
Name: count, dtype: int64

Saved submission: submission_improved.csv

=== CHECK SAVED FILE ===
Loaded file rows: 519